In [5]:
import os
import json

data = []

def extract_data_from_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read().strip()
        return [float(line) for line in content.split('\n')]

def extract_data_from_json(file_path):
    with open(file_path, 'r') as file:
        data_dict = json.load(file)
        return data_dict

def split_method_mahalanobis(input_string):
    # Split the string using "_" as the delimiter
    split_parts = input_string.split("@")

    # Use list comprehension to filter out substrings
    desired_substrings = [part for part in split_parts if "@" not in part]

    method = desired_substrings[0].split("_")[0]
    mahalanobis_method = desired_substrings[0].split("_")[1]
    beta = desired_substrings[0].split("_")[3]

    model = desired_substrings[1]

    proposal_model = desired_substrings[2]
    dim_red = desired_substrings[3].split("_")[0]
    dim_red_component = desired_substrings[3].split("_")[1]

    doc = {
        "method":method,
        "mahalanobis_method": mahalanobis_method,
        "beta": beta,
        "model": model,
        "proposal_model": proposal_model,
        "dim_red": dim_red,
        "n_components": dim_red_component,
    }
    return doc 


def split_method_samAlone(input_string):
    # Split the string using "_" as the delimiter
    split_parts = input_string.split("@")

    # Use list comprehension to filter out substrings
    desired_substrings = [part for part in split_parts if "@" not in part]

    method = desired_substrings[0]
    model_proposal = desired_substrings[1]

    doc = {
        "method":method,
        "proposal_model": model_proposal,
    }
    return doc 

def split_method_others(input_string):
    # Split the string using "_" as the delimiter
    split_parts = input_string.split("@")

    # Use list comprehension to filter out substrings
    desired_substrings = [part for part in split_parts if "@" not in part]

    method = desired_substrings[0]
    model_proposal = desired_substrings[2]
    model = desired_substrings[1]

    doc = {
        "method":method,
        "proposal_model": model_proposal,
        "model": model,
    }
    return doc 

def extract_data_from_file_json(file_path):
    with open(file_path, 'r') as file:
        content = file.read().strip()
        return json.loads(content)



def process_directory(directory_path):

    data = []
    for seed_folder in os.listdir(directory_path):
        seed_path = os.path.join(directory_path, seed_folder)
        
        if os.path.isdir(seed_path):
            seed_number = seed_folder.replace("seed", "")
            
            for label_folder in os.listdir(seed_path):
                label_path = os.path.join(seed_path, label_folder)
                
                if os.path.isdir(label_path) and label_folder.endswith("_100"):
                    label_opt = label_folder.split("_") 
                    label = label_opt[0]
                    unlabeled = label_opt[1]

                    #print(label)
                    for method_folder in os.listdir(label_path):
                        method_path = os.path.join(label_path, method_folder)
                        #print(method_folder)

                        if "fewshotMahalanobis" in method_folder and os.path.isdir(method_path):
                            doc_fewshotmahalanobis = split_method_mahalanobis(method_folder)

                            mAP_test = "-"
                            result_path = os.path.join(method_path, 'mAP_fewshotMahalanobis.txt')
                            if os.path.exists(result_path):
                                mAP_test = extract_data_from_file(result_path)[1]

                            val_result_path = os.path.join(method_path, 'mAP_val_fewshotMahalanobis.txt')
                            mAP_val = "-"
                            if os.path.exists(val_result_path):
                                mAP_val = extract_data_from_file(val_result_path)[1]
                                

                            doc_fewshotmahalanobis["mAP_test"] = mAP_test
                            doc_fewshotmahalanobis["mAP_val"] = mAP_val

                            test_json = os.path.join(method_path, 'stats_fewshotMahalanobis.json')
                            val_json = os.path.join(method_path, 'stats_val_fewshotMahalanobis.json')
                            stats_json = os.path.join(method_path, 'stats.json')

                            if os.path.exists(test_json):
                                stats_data = extract_data_from_json(test_json)
                                amount_imgs_test = stats_data.get("amount_imgs")
                                ground_truth_ids_test = stats_data.get("ground_truth_ids")

                                doc_fewshotmahalanobis["Test"] = amount_imgs_test
                                doc_fewshotmahalanobis["Pineapple test"] = ground_truth_ids_test
                            
                            if os.path.exists(val_json):

                                stats_data = extract_data_from_json(val_json)
                                amount_imgs_val = stats_data.get("amount_imgs")
                                ground_truth_ids_val = stats_data.get("ground_truth_ids")

                                doc_fewshotmahalanobis["Val"] = amount_imgs_val
                                doc_fewshotmahalanobis["Pineapple Validation"] = ground_truth_ids_val

                            if os.path.exists(stats_json):
                                stats_data = extract_data_from_json(stats_json)
                                doc_fewshotmahalanobis["dimension"] = stats_data.get("dimension")
                                doc_fewshotmahalanobis["background"] = stats_data.get("all")
                                doc_fewshotmahalanobis["context"] = stats_data.get("context")
                                doc_fewshotmahalanobis["Pineapple labeled"] = stats_data.get("labeled")
                                if "positive_definite" in doc_fewshotmahalanobis and "semi_positive_definite" in doc_fewshotmahalanobis:
                                    doc_fewshotmahalanobis["positive_definite"] = stats_data.get("positive_definite")
                                    doc_fewshotmahalanobis["semi_positive_definite"] = stats_data.get("semi_positive_definite")

                            doc_fewshotmahalanobis["seed_number"] = seed_number
                            doc_fewshotmahalanobis["label"] = label
                            doc_fewshotmahalanobis["unlabeled"] = unlabeled
                            data.append(doc_fewshotmahalanobis)

                        if "samAlone" in method_folder and os.path.isdir(method_path):
                            doc = split_method_samAlone(method_folder)

                            mAP_test = "-"
                            result_path = os.path.join(method_path, 'mAP_samAlone.txt')
                            if os.path.exists(result_path):
                                mAP_test = extract_data_from_file(result_path)[1]

                            val_result_path = os.path.join(method_path, 'mAP_val_samAlone.txt')
                            mAP_val = "-"
                            if os.path.exists(val_result_path):
                                mAP_val = extract_data_from_file(val_result_path)[1]
                                

                            doc["mAP_test"] = mAP_test
                            doc["mAP_val"] = mAP_val

                            test_json = os.path.join(method_path, 'stats_samAlone.json')
                            val_json = os.path.join(method_path, 'stats_val_samAlone.json')

                            
                            if os.path.exists(test_json):

                                stats_data = extract_data_from_file_json(test_json)
                                amount_imgs_test = stats_data.get("amount_imgs")
                                ground_truth_ids_test = stats_data.get("ground_truth_ids")

                                doc["Test"] = amount_imgs_test
                                doc["Pineapple test"] = ground_truth_ids_test

                            elif os.path.exists(val_json):

                                stats_data = extract_data_from_file_json(val_json)
                                amount_imgs_val = stats_data.get("amount_imgs")
                                ground_truth_ids_val = stats_data.get("ground_truth_ids")

                                doc["Val"] = amount_imgs_val
                                doc["Pineapple Validation"] = ground_truth_ids_val

                            doc["seed_number"] = seed_number
                            doc["label"] = label
                            doc["unlabeled"] = unlabeled
                            data.append(doc)


                        if "fewshotOOD" in method_folder and os.path.isdir(method_path):
                            doc_fewshotmahalanobis = split_method_others(method_folder)

                            mAP_test = "-"
                            result_path = os.path.join(method_path, 'mAP_fewshotOOD_std1.txt')
                            if os.path.exists(result_path):
                                mAP_test = extract_data_from_file(result_path)[1]

                            val_result_path = os.path.join(method_path, 'mAP_val_fewshotOOD_std1.txt')
                            mAP_val = "-"
                            if os.path.exists(val_result_path):
                                mAP_val = extract_data_from_file(val_result_path)[1]
                                
                            doc_fewshotmahalanobis["mAP_test"] = mAP_test
                            doc_fewshotmahalanobis["mAP_val"] = mAP_val

                            test_json = os.path.join(method_path, 'stats_fewshotODD.json')
                            val_json = os.path.join(method_path, 'stats_val_fewshotODD.json')

                            if os.path.exists(test_json):
                                stats_data = extract_data_from_json(test_json)
                                amount_imgs_test = stats_data.get("amount_imgs")
                                ground_truth_ids_test = stats_data.get("ground_truth_ids")

                                doc_fewshotmahalanobis["Test"] = amount_imgs_test
                                doc_fewshotmahalanobis["Pineapple test"] = ground_truth_ids_test
                            
                            if os.path.exists(val_json):

                                stats_data = extract_data_from_json(val_json)
                                amount_imgs_val = stats_data.get("amount_imgs")
                                ground_truth_ids_val = stats_data.get("ground_truth_ids")

                                doc_fewshotmahalanobis["Val"] = amount_imgs_val
                                doc_fewshotmahalanobis["Pineapple Validation"] = ground_truth_ids_val

                            doc_fewshotmahalanobis["seed_number"] = seed_number
                            doc_fewshotmahalanobis["label"] = label
                            doc_fewshotmahalanobis["unlabeled"] = unlabeled
                            data.append(doc_fewshotmahalanobis)
                        
                        if "fewshotBDCSPN" in method_folder and os.path.isdir(method_path):
                            doc_fewshotmahalanobis = split_method_others(method_folder)

                            mAP_test = "-"
                            result_path = os.path.join(method_path, 'mAP_fewshotBDCSPN.txt')
                            if os.path.exists(result_path):
                                mAP_test = extract_data_from_file(result_path)[1]

                            val_result_path = os.path.join(method_path, 'mAP_val_fewshotBDCSPN.txt')
                            mAP_val = "-"
                            if os.path.exists(val_result_path):
                                mAP_val = extract_data_from_file(val_result_path)[1]
                                
                            doc_fewshotmahalanobis["mAP_test"] = mAP_test
                            doc_fewshotmahalanobis["mAP_val"] = mAP_val

                            test_json = os.path.join(method_path, 'stats_fewshotBDCSPN.json')
                            val_json = os.path.join(method_path, 'stats_val_fewshotBDCSPN.json')

                            if os.path.exists(test_json):
                                stats_data = extract_data_from_json(test_json)
                                amount_imgs_test = stats_data.get("amount_imgs")
                                ground_truth_ids_test = stats_data.get("ground_truth_ids")

                                doc_fewshotmahalanobis["Test"] = amount_imgs_test
                                doc_fewshotmahalanobis["Pineapple test"] = ground_truth_ids_test
                            
                            if os.path.exists(val_json):

                                stats_data = extract_data_from_json(val_json)
                                amount_imgs_val = stats_data.get("amount_imgs")
                                ground_truth_ids_val = stats_data.get("ground_truth_ids")

                                doc_fewshotmahalanobis["Val"] = amount_imgs_val
                                doc_fewshotmahalanobis["Pineapple Validation"] = ground_truth_ids_val

                            doc_fewshotmahalanobis["seed_number"] = seed_number
                            doc_fewshotmahalanobis["label"] = label
                            doc_fewshotmahalanobis["unlabeled"] = unlabeled
                            data.append(doc_fewshotmahalanobis)

                        if "fewshot1" in method_folder and os.path.isdir(method_path):
                            doc_fewshotmahalanobis = split_method_others(method_folder)

                            mAP_test = "-"
                            result_path = os.path.join(method_path, 'mAP_fewshot1_std2.txt')
                            if os.path.exists(result_path):
                                mAP_test = extract_data_from_file(result_path)[1]

                            val_result_path = os.path.join(method_path, 'mAP_val_fewshot1_std2.txt')
                            mAP_val = "-"
                            if os.path.exists(val_result_path):
                                mAP_val = extract_data_from_file(val_result_path)[1]
                                
                            doc_fewshotmahalanobis["mAP_test"] = mAP_test
                            doc_fewshotmahalanobis["mAP_val"] = mAP_val

                            test_json = os.path.join(method_path, 'stats_fewshot1.json')
                            val_json = os.path.join(method_path, 'stats_val_fewshot1.json')

                            if os.path.exists(test_json):
                                stats_data = extract_data_from_json(test_json)
                                amount_imgs_test = stats_data.get("amount_imgs")
                                ground_truth_ids_test = stats_data.get("ground_truth_ids")

                                doc_fewshotmahalanobis["Test"] = amount_imgs_test
                                doc_fewshotmahalanobis["Pineapple test"] = ground_truth_ids_test
                            
                            if os.path.exists(val_json):

                                stats_data = extract_data_from_json(val_json)
                                amount_imgs_val = stats_data.get("amount_imgs")
                                ground_truth_ids_val = stats_data.get("ground_truth_ids")

                                doc_fewshotmahalanobis["Val"] = amount_imgs_val
                                doc_fewshotmahalanobis["Pineapple Validation"] = ground_truth_ids_val

                            doc_fewshotmahalanobis["seed_number"] = seed_number
                            doc_fewshotmahalanobis["label"] = label
                            doc_fewshotmahalanobis["unlabeled"] = unlabeled
                            data.append(doc_fewshotmahalanobis)
    return data
                                



import pandas as pd

def main():
    directory_path = 'F:/PINEAPPLES2_5mts_nosplits_ALL_METHODS/PINEAPPLES2_5mts_nosplits_ALL_METHODS'
    data = process_directory(directory_path)

    # Print or use the collected data as needed
    #for entry in data:
    #    print(json.dumps(entry, indent=2))
    # Create an empty DataFrame
    df = pd.DataFrame()
    dfs = []

    # Iterate through the list of JSON objects
    for json_obj in data:
        # Convert each JSON object to a Series and append it to the DataFrame
        print(json_obj)
        df = pd.DataFrame([json_obj])
        dfs.append(df)
    result_df = pd.concat(dfs, ignore_index=True)

    # Display the resulting DataFrame
    return result_df

#if __name__ == '__main__':
#    main()
result_df = main()
result_df.to_csv("result_part_edgesam.csv")

{'method': 'fewshot1', 'proposal_model': 'edgesam', 'model': 'swinv2_base_window8_256.ms_in1k', 'mAP_test': 0.11258145254713706, 'mAP_val': 0.09335176852987209, 'Test': 100, 'Pineapple test': 4650, 'Val': 4, 'Pineapple Validation': 194, 'seed_number': '0', 'label': '10', 'unlabeled': '100'}
{'method': 'fewshot1', 'proposal_model': 'fastsam', 'model': 'swinv2_base_window8_256.ms_in1k', 'mAP_test': 0.38281481863409855, 'mAP_val': 0.31822435653885556, 'Test': 100, 'Pineapple test': 4650, 'Val': 4, 'Pineapple Validation': 194, 'seed_number': '0', 'label': '10', 'unlabeled': '100'}
{'method': 'fewshot1', 'proposal_model': 'mobilesam', 'model': 'swinv2_base_window8_256.ms_in1k', 'mAP_test': 0.2542984495134782, 'mAP_val': 0.25518397878143206, 'Test': 100, 'Pineapple test': 4650, 'Val': 4, 'Pineapple Validation': 194, 'seed_number': '0', 'label': '10', 'unlabeled': '100'}
{'method': 'fewshot1', 'proposal_model': 'sam', 'model': 'swinv2_base_window8_256.ms_in1k', 'mAP_test': 0.3182967868142794,